In [1]:
# Day 7 — SQL Filtering & Aggregation (COUNT, AVG, GROUP BY)
#**Goal:** practice `COUNT`, `AVG`, `GROUP BY`, `WHERE`, `HAVING`, `ORDER BY`  
#**Example:** average nitrate by county


In [2]:
#Python: imports and DB config

import os
import pandas as pd
from sqlalchemy import create_engine, text

# ---- Connection config (edit if your Docker ports differ) ----
PG_USER = os.getenv("PGUSER", "env_user")
PG_PASS = os.getenv("PGPASSWORD", "env_pass")
PG_HOST = os.getenv("PGHOST", "localhost")
PG_PORT = int(os.getenv("PGPORT", "5433"))
PG_DB = os.getenv("PGDATABASE", "envdb")

engine = create_engine(f"postgresql+psycopg2://{PG_USER}:{PG_PASS}@{PG_HOST}:{PG_PORT}/{PG_DB}")

#Target table 
SCHEMA = "water_quality"
TABLE = "measurements"
FULL_TABLE = f"{SCHEMA}.{TABLE}"


In [3]:
#Python: create schema + create a csv with data for practice
USE_DEMO_DATA = True  # <- set to False if you already have your dataset loaded

with engine.begin() as conn:
    if USE_DEMO_DATA:
        # Create schema and table if not exists
        conn.execute(text(f"CREATE SCHEMA IF NOT EXISTS {SCHEMA};"))
        conn.execute(text(f"""
            CREATE TABLE IF NOT EXISTS {FULL_TABLE} (
                id SERIAL PRIMARY KEY,
                sample_date date,
                county text,
                station_id text,
                nitrate_mgl numeric,
                geom geometry(Point, 4326)
            );
        """))
        # Clean any prior demo rows (so the cell is re-runnable)
        conn.execute(text(f"DELETE FROM {FULL_TABLE};"))

        import pandas as pd

# --- Create demo dataset ---
data = {
    "sample_date": [
        "2025-09-28", "2025-09-29", "2025-10-01",
        "2025-09-27", "2025-10-02", "2025-10-03",
        "2025-09-30", "2025-10-01"
    ],
    "county": [
        "Clatsop", "Clatsop", "Clatsop",
        "Tillamook", "Tillamook", "Tillamook",
        "Lincoln", "Lincoln"
    ],
    "station_id": [
        "ST-001", "ST-002", "ST-001",
        "ST-010", "ST-011", "ST-012",
        "ST-020", "ST-021"
    ],
    "nitrate_mgl": [
        1.5, 2.1, None,
        3.2, 2.8, 4.0,
        0.9, 1.1
    ],
    "longitude": [
        -123.9, -123.8, -123.9,
        -123.8, -123.7, -123.6,
        -124.0, -124.1
    ],
    "latitude": [
        46.16, 46.18, 46.16,
        45.45, 45.42, 45.40,
        44.95, 44.97
    ]
}

df = pd.DataFrame(data)

# --- Save to CSV ---
csv_path = os.path.join("csv", "water_quality_measurements.csv")
df.to_csv(csv_path, index=False)

print(f"CSV file saved: {csv_path}")
df.head()

CSV file saved: csv\water_quality_measurements.csv


,sample_date,county,station_id,nitrate_mgl,longitude,latitude
0,2025-09-28,Clatsop,ST-001,1.5,-123.9,46.16
1,2025-09-29,Clatsop,ST-002,2.1,-123.8,46.18
2,2025-10-01,Clatsop,ST-001,NaN,-123.9,46.16
3,2025-09-27,Tillamook,ST-010,3.2,-123.8,45.45
4,2025-10-02,Tillamook,ST-011,2.8,-123.7,45.42


In [4]:
#Load CSV created above
df_loaded = pd.read_csv("csv/water_quality_measurements.csv")
print("✅ CSV loaded successfully!")
df_loaded.info()
df_loaded.head()


✅ CSV loaded successfully!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sample_date  8 non-null      object 
 1   county       8 non-null      object 
 2   station_id   8 non-null      object 
 3   nitrate_mgl  7 non-null      float64
 4   longitude    8 non-null      float64
 5   latitude     8 non-null      float64
dtypes: float64(3), object(3)
memory usage: 516.0+ bytes


,sample_date,county,station_id,nitrate_mgl,longitude,latitude
0,2025-09-28,Clatsop,ST-001,1.5,-123.9,46.16
1,2025-09-29,Clatsop,ST-002,2.1,-123.8,46.18
2,2025-10-01,Clatsop,ST-001,NaN,-123.9,46.16
3,2025-09-27,Tillamook,ST-010,3.2,-123.8,45.45
4,2025-10-02,Tillamook,ST-011,2.8,-123.7,45.42


In [ ]:
#Load CSV to Postgres/PostGIS database (subsequent pd.read_sql() functions pull from this data in Docker PostGIS DB)
df_loaded.to_sql(
    "measurements",
    engine,
    schema="water_quality",
    if_exists="replace",
    index=False
)

8

In [40]:
#Python sanity check (peek + schema)

import pandas as pd

#Peek first rows
preview = pd.read_sql(f"SELECT * FROM {FULL_TABLE} ORDER BY 1 LIMIT 10;", engine)

#Inspect column types as seen by Postgres
cols = pd.read_sql("""
    SELECT column_name, data_type
    FROM information_schema.columns
    WHERE table_schema = '{SCHEMA}' AND table_name = '{TABLE}'
    ORDER BY ordinal_position;
""", engine)

display(preview)
display(cols)

,sample_date,county,station_id,nitrate_mgl,longitude,latitude,geom
0,2025-09-27,Tillamook,ST-010,3.2,-123.8,45.45,0101000020E61000003333333333F35EC09A99999999B9...
1,2025-09-28,Clatsop,ST-001,1.5,-123.9,46.16,0101000020E61000009A99999999F95EC014AE47E17A14...
2,2025-09-29,Clatsop,ST-002,2.1,-123.8,46.18,0101000020E61000003333333333F35EC0D7A3703D0A17...
3,2025-09-30,Lincoln,ST-020,0.9,-124.0,44.95,0101000020E61000000000000000005FC09A9999999979...
4,2025-10-01,Lincoln,ST-021,1.1,-124.1,44.97,0101000020E61000006666666666065FC05C8FC2F5287C...
5,2025-10-01,Clatsop,ST-001,NaN,-123.9,46.16,0101000020E61000009A99999999F95EC014AE47E17A14...
6,2025-10-02,Tillamook,ST-011,2.8,-123.7,45.42,0101000020E6100000CDCCCCCCCCEC5EC0F6285C8FC2B5...
7,2025-10-03,Tillamook,ST-012,4.0,-123.6,45.40,0101000020E61000006666666666E65EC03333333333B3...


,column_name,data_type


In [ ]:
#Since cols displayed no data, the below is some more in-depth diagnostic code to try and determine why
#Changed lookup-filter/query used to try and avoid mismatches(often search_path/casing quirks)
import pandas as pd

# List all tables in the schema
tables = pd.read_sql(f"""
    SELECT table_schema, table_name
    FROM information_schema.tables
    WHERE table_schema = '{SCHEMA}'
    ORDER BY table_name;
""", engine)

# Robust column introspection using pg_catalog
cols = pd.read_sql(f"""
    SELECT
        a.attname AS column_name,
        format_type(a.atttypid, a.atttypmod) AS data_type,
        a.attnum AS ordinal_position
    FROM pg_attribute a
    WHERE a.attrelid = '{FULL_TABLE}'::regclass
      AND a.attnum > 0 AND NOT a.attisdropped
    ORDER BY a.attnum;
""", engine)

display(tables)
display(cols)


,table_schema,table_name
0,water_quality,measurements


,column_name,data_type,ordinal_position
0,sample_date,text,1
1,county,text,2
2,station_id,text,3
3,nitrate_mgl,double precision,4
4,longitude,double precision,5
5,latitude,double precision,6


In [ ]:
#COUNT(*) and basic WHERE filters
total_rows = pd.read_sql(f"SELECT COUNT(*) AS n FROM {FULL_TABLE};", engine)

#Count rows with nitrate value(exclude NULLs)
non_null_count = pd.read_sql(f"""
    SELECT COUNT(*) AS n
    FROM {FULL_TABLE}
    WHERE nitrate_mgl IS NOT NULL;
""", engine)

display(total_rows)
display(non_null_count)

,n
0,8


,n
0,7


In [19]:
#COUNT with DISTINCT (e.g. distinct stations by county)

#Distinct station count overall
stations_overall = pd.read_sql(f"""
    SELECT COUNT(DISTINCT station_id) AS distinct_stations
    FROM {FULL_TABLE};
""", engine)

#Distinct stations per county
stations_by_county = pd.read_sql(f"""
SELECT county, COUNT(DISTINCT station_id) AS distinct_stations
FROM {FULL_TABLE}
GROUP BY county
ORDER BY county;
""", engine)


display(stations_overall)
display(stations_by_county)

,distinct_stations
0,7


,county,distinct_stations
0,Clatsop,2
1,Lincoln,2
2,Tillamook,3


In [ ]:
#AVG basics + rounding + NULL handling

# Average across all rows (AVG ignores NULLs)
avg_all = pd.read_sql(f"""
SELECT ROUND(AVG(nitrate_mgl)::numeric, 3) AS avg_nitrate_mgl 
FROM {FULL_TABLE};
""", engine)

#Average for a single county filter (SELECT 'Clatsop' AS county means grab the 'county' column and only 'Catslop' for results table , WHERE county = 'Clatsop' tells which rows to include in the calcuation)
avg_clatsop = pd.read_sql(f"""
SELECT 'Clatsop' AS county,
    ROUND(AVG(nitrate_mgl)::numeric, 3) AS avg_nitrate_mgl
    FROM {FULL_TABLE}
WHERE county = 'Clatsop';
""", engine)

display(avg_all)
display(avg_clatsop)

,avg_nitrate_mgl
0,2.229


,county,avg_nitrate_mgl
0,Clatsop,1.8


In [ ]:
#GROUP BY county (average nitrate by county) *SELECT county means grabe the county column from FULL_TABLE and put it in results table (prior to further filters being applied)
avg_by_county = pd.read_sql(f"""
SELECT county,
    ROUND(AVG(nitrate_mgl)::numeric, 3) AS avg_nitrate_mgl,
    COUNT(*) AS sample_count
FROM {FULL_TABLE}
WHERE nitrate_mgl IS NOT NULL
GROUP BY county
ORDER BY county;
""", engine)

avg_by_county

,county,avg_nitrate_mgl,sample_count
0,Clatsop,1.800,2
1,Lincoln,1.000,2
2,Tillamook,3.333,3


In [30]:
#Add a date filter (WHERE) and a minimum sample threshold (HAVING)
avg_recent = pd.read_sql(f"""
SELECT county,
    ROUND(AVG(nitrate_mgl)::numeric, 3) AS avg_nitrate_mgl,
    COUNT(*) AS sample_count
FROM {FULL_TABLE}
WHERE nitrate_mgl IS NOT NULL
    AND sample_date::date >= DATE '2025-09-28'
GROUP BY county
HAVING COUNT(*) >= 2
ORDER BY avg_nitrate_mgl DESC;  --highest avg first
""", engine)

avg_recent

,county,avg_nitrate_mgl,sample_count
0,Tillamook,3.4,2
1,Clatsop,1.8,2
2,Lincoln,1.0,2


In [32]:
#Top/bottom N counties (ORDER BY + LIMIT)
#Top 3 by average nitrate
top3 = pd.read_sql(f"""
SELECT county,
    ROUND(AVG(nitrate_mgl)::numeric, 3) AS avg_nitrate_mgl,
    COUNT(*) AS sample_count
FROM {FULL_TABLE}
WHERE nitrate_mgl IS NOT NULL
GROUP BY county
ORDER BY avg_nitrate_mgl DESC
LIMIT 3;
""", engine)

#Bottom 3 by average nitrate
bottom3 = pd.read_sql(f"""
SELECT county,
    ROUND(AVG(nitrate_mgl)::numeric, 3) AS avg_nitrate_mgl,
    COUNT(*) AS sample_count
FROM {FULL_TABLE}
WHERE nitrate_mgl IS NOT NULL
GROUP BY county
ORDER BY avg_nitrate_mgl ASC
LIMIT 3;
""", engine)

display(top3)
display(bottom3)

,county,avg_nitrate_mgl,sample_count
0,Tillamook,3.333,3
1,Clatsop,1.800,2
2,Lincoln,1.000,2


,county,avg_nitrate_mgl,sample_count
0,Lincoln,1.000,2
1,Clatsop,1.800,2
2,Tillamook,3.333,3


In [36]:
#Filter by multiple counties (IN/ANY)

from sqlalchemy import text

subset = pd.read_sql(
    text(f"""
        SELECT county,
            ROUND(AVG(nitrate_mgl)::numeric, 3) AS avg_nitrate_mgl,
            COUNT(*) AS sample_count
        FROM {FULL_TABLE}
        WHERE nitrate_mgl IS NOT NULL
            AND county = ANY(:counties)
        GROUP BY county
        ORDER BY county;
         """),
        engine,
        params={"counties": ["Clatsop", "Tillamook"]}
)

subset

,county,avg_nitrate_mgl,sample_count
0,Clatsop,1.800,2
1,Tillamook,3.333,3


In [38]:
#Export aggregated results (CSV)

out = pd.read_sql(f"""
    SELECT county,
        ROUND(AVG(nitrate_mgl)::numeric, 3) AS avg_nitrate_mgl,
        COUNT(*) AS sample_count
    FROM {FULL_TABLE}
WHERE nitrate_mgl IS NOT NULL
GROUP BY county
ORDER BY county;
""", engine)

out_path = os.path.join("csv", "avg_nitrate_by_county.csv")
out.to_csv(out_path, index=False)
print("Saved:", out_path)
out.head()

Saved: csv\avg_nitrate_by_county.csv


,county,avg_nitrate_mgl,sample_count
0,Clatsop,1.800,2
1,Lincoln,1.000,2
2,Tillamook,3.333,3


In [ ]:
#Optional: includes our lat/lon data as a geom column in WKB (well-known binary) a long binary text that encodes for the lat/lon
from sqlalchemy import text

with engine.begin() as conn:
    conn.execute(text(f"ALTER TABLE {FULL_TABLE} DROP COLUMN IF EXISTS geom;"))
    conn.execute(text(f"ALTER TABLE {FULL_TABLE} ADD COLUMN geom geometry(Point, 4326);"))
    conn.execute(text(f"""
        UPDATE {FULL_TABLE}
        SET geom = CASE
            WHEN longitude IS NOT NULL AND latitude IS NOT NULL
            THEN ST_SetSRID(ST_MakePoint(longitude::double precision, latitude::double precision), 4326)
            ELSE NULL
        END;
    """))


In [ ]:
#Optional: displays our lat/lon as lat/lon in a table, rather than as WKB
import pandas as pd

geom_preview = pd.read_sql(f"""
    SELECT
        sample_date,
        county,
        station_id,
        nitrate_mgl,
        ST_AsText(geom) AS geom_text
    FROM {FULL_TABLE}
    ORDER BY county, sample_date
    LIMIT 10;
""", engine)

geom_preview


,sample_date,county,station_id,nitrate_mgl,geom_text
0,2025-09-28,Clatsop,ST-001,1.5,POINT(-123.9 46.16)
1,2025-09-29,Clatsop,ST-002,2.1,POINT(-123.8 46.18)
2,2025-10-01,Clatsop,ST-001,NaN,POINT(-123.9 46.16)
3,2025-09-30,Lincoln,ST-020,0.9,POINT(-124 44.95)
4,2025-10-01,Lincoln,ST-021,1.1,POINT(-124.1 44.97)
5,2025-09-27,Tillamook,ST-010,3.2,POINT(-123.8 45.45)
6,2025-10-02,Tillamook,ST-011,2.8,POINT(-123.7 45.42)
7,2025-10-03,Tillamook,ST-012,4.0,POINT(-123.6 45.4)
